<a href="https://colab.research.google.com/github/ms624atyale/NLP_2025/blob/main/12_TextScrapingwithoutHTMLTags_Saving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color = 'red'> 🐹 👀 🐾 **Text/Content/Web Scraping without HTML tags**

## **API-based Data Collection**

### <font color = 'blue'> **cf., Crawling (a.k.a. HTML Scraping) or Text Mining**

In [ ]:
pip install requests

In [ ]:
import requests #Import the requests library to make HTTP requests.

def get_wikipedia_page(title):                   #Define a function
    URL = "https://en.wikipedia.org/w/api.php"  #Set the API(application program interface) endpoint URL: https://en.wikipedia.org/w/api.php.

    PARAMS = {                                  #Build PARAMS (query parameters) for the API request:
        "action": "query",                      #ask the API to run a query
        "format": "json",                       #request a JSON response
        "prop": "extracts",                     #ask for the page extract (clean text summary)
        "titles": title,                        #specify which page to fetch (by title)
        "explaintext": 1                        #return plain text (no HTML/markup)

    }

    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 " #header to mimic a normal browser request (helps avoid blocks)
                      "(KHTML, like Gecko) Chrome/123.0 Safari/537.36"
    }

    response = requests.get(URL, params=PARAMS, headers=headers)           #Send a GET request to the API with requests

    if response.status_code != 200:                                        #Check the HTTP status code: If not 200 OK, print an error message and return None.
        print("HTTP error:", response.status_code)
        return None

    try:
        data = response.json()                                            #Try to parse the response body as JSON with response.json():
    except:
        print("JSON decode error")                                        #If JSON decoding fails, print a debug message showing the start of the raw response and return None.
        print("Raw response:", response.text[:500])
        return None

    pages = data.get("query", {}).get("pages", {})                      #Navigate the JSON structure to the page data: data["query"]["pages"] (a dictionary keyed by numeric page id).
    page = next(iter(pages.values()))                                   #Extract the single page object with next(iter(pages.values())) (handles the unknown page id).
    return page.get("extract", "")                                      #Return the page’s plain-text extract via page.get("extract", "").
                                                                        #If the page exists, this is the article text; if not, it returns an empty string (or None earlier if errors occurred).

In [ ]:
text = get_wikipedia_page("KPop Demon Hunters")
print(text[:500])                                 #Forward indexing (index 1 to index 499: 500 characters including symbols and white space)

KPop Demon Hunters is a 2025 American animated musical urban fantasy film directed by Maggie Kang and Chris Appelhans from a screenplay they co-wrote with Danya Jimenez and Hannah McMechan, based on a story conceived by Kang. Produced by Sony Pictures Animation for Netflix, the film stars the voices of Arden Cho, Ahn Hyo-seop, May Hong, Ji-young Yoo, Yunjin Kim, Daniel Dae Kim, Ken Jeong, and Lee Byung-hun. The film follows a K-pop girl group, Huntr/x, who lead double lives as demon hunters; the


In [ ]:
titles = [
    "K-pop",
    "Korean Wave",
    "KPop Demon Hunters",
    "Hybe",
    "BTS",
    "2024 Nobel Prize in Literature",
    "Han Kang",
    "Bong Joon Ho",
    "Pachinko",
    "Minjung Son"
]

corpus = {}

for t in titles:
    txt = get_wikipedia_page(t)
    if txt:
        corpus[t] = txt
    else:
        print("Failed:", t)

# Show first 200 chars for each
for title, text in corpus.items():
    print("\n====", title, "====")
    print(text[:200])

Failed: Minjung Son

==== K-pop ====
K-pop (Korean: 케이팝; RR: Keipap; an abbreviation of "Korean popular music") is a form of popular music originating in South Korea. The music genre that the term is used to refer to colloquially emerged

==== Korean Wave ====
The Korean Wave, or hallyu (Korean: 한류; ), is the dramatic rise in global interest in South Korean popular culture since the 1990s—led by K-pop, K-dramas, and films, with keystone successes including 

==== KPop Demon Hunters ====
KPop Demon Hunters is a 2025 American animated musical urban fantasy film directed by Maggie Kang and Chris Appelhans from a screenplay they co-wrote with Danya Jimenez and Hannah McMechan, based on a

==== Hybe ====
Hybe Co., Ltd. (Korean: 하이브; haibeu), commonly known as simply Hybe, is a South Korean multinational entertainment company established in 2005 by Bang Si-hyuk as Big Hit Entertainment Co., Ltd.
The co

==== BTS ====
BTS (Korean: 방탄소년단; RR: Bangtan Sonyeondan; lit. Bulletproof Boy Scouts), a

✅ Script A — create one TXT file per title

- All scripts use Wikipedia API

In [ ]:
import os

os.makedirs("wiki_txts", exist_ok=True)

for title in titles:
    txt = get_wikipedia_page(title)
    if not txt:
        print(f"Skipping: {title}")
        continue

    fname = title.replace(" ", "_").replace("'", "") + ".txt"
    path = os.path.join("wiki_txts", fname)

    with open(path, "w", encoding="utf-8") as f:
        f.write(txt)

    print(f"Saved: {path}")

Saved: wiki_txts/K-pop.txt
Saved: wiki_txts/Korean_Wave.txt
Saved: wiki_txts/KPop_Demon_Hunters.txt
Saved: wiki_txts/Hybe.txt
Saved: wiki_txts/BTS.txt
Saved: wiki_txts/2024_Nobel_Prize_in_Literature.txt
Saved: wiki_txts/Han_Kang.txt
Saved: wiki_txts/Bong_Joon_Ho.txt
Saved: wiki_txts/Pachinko.txt
Skipping: Minjung Son


✅ Script B — Create one TXT file with records separated by @

In [ ]:
output = []

for title in titles:
    txt = get_wikipedia_page(title)
    if not txt:
        txt = ""   # store empty if missing
    block = f"@@@@@\nTITLE: {title}\n{txt}\n"
    output.append(block)

final = "\n".join(output)

with open("wiki_corpus_delimited.txt", "w", encoding="utf-8") as f:
    f.write(final)

print("Saved: wiki_corpus_delimited.txt")

Saved: wiki_corpus_delimited.txt


✅ Script C — Create one CSV with two columns (title + text)

#🐹 🐾 📌 **Use this!!!**📌

In [ ]:
import csv

rows = []

for title in titles:
    txt = get_wikipedia_page(title)
    rows.append([title, txt])

with open("wiki_corpus.csv", "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["title", "text"])
    writer.writerows(rows)

print("Saved: wiki_corpus.csv")

Saved: wiki_corpus.csv


# <font color = 'green'>🐹 🐾 **Gutenberg project [by downloading]**

#📕 **Children's Picture Books**
###[**Project Gutenberg**](https://gutenberg.org/)
- **Beatrix Potter: Search by 'Beatrix Potter'**
    - **The Tale of Peter Rabit**
    - **The Tale of Benjamin Bunny**
    - **The Tale of Jemima Puddle-Duck**
    - **The Tale of Mrs. Tiggy-Winkle**
    - **The Tale of Squirrel Nutkin**
    - **The Tale of Tom Kitten**

- **Leslie Brooke: Search by 'Leslie Brooke'**
    - **The Tailor and the Crow**
    - **The Golden Goose Book**
    - **Jonny Crow's Garden**
    - **A Nursery Rhyme Picture Book**

##**Step 1**

- **For each volume, take a look at a published version with the tab, "READ NOW."**
- **Download UTF-8 on your machine**
- **Open your 메모장 or TextEdit.**
- **Save it as plain text.**

## **Step 2**
- Make a folder on your Github repository with a new name "Data_Plain"
- Upload

# <font color = 'red'> 🐹🐾 **Final Script to prepare input text for further analysis (e.g., Common Core Words, Wordcloud, Lexical Diversity, etc.)**

  - # <font color = 'blue'> 🐹🐾 **Important & Useful!**
  - ### **This script will be based on plain text for 10 volumes above.**